# Transformer
## Easy to understand implementation of the Transformer Architecture

In [50]:
import torch

In [51]:
class PositionalEncoder(torch.nn.Module):
    def __init__(self, max_seq_len, d_model):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.d_model = d_model

        self.pe = torch.zeros(self.max_seq_len, self.d_model)

        # positional encoder 
        for j in range(max_seq_len):
            for i in range(0, d_model):
                if i%2 == 0:
                    k = i//2
                    wk = 1 / (10000**((2*k)/d_model))
                    self.pe[j][i] = torch.sin(torch.tensor(wk*j))
                else:
                    self.pe[j][i] = torch.cos(torch.tensor(wk*j))

        self.pe = self.pe.detach()

    def show_pe(self):
        print(self.pe)

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(0)

        if x.shape[1]>self.max_seq_len:
            raise Exception("Number of tokens exceeds max_seq_len")
        
        if x.shape[2] != self.d_model:
            raise Exception("Token dimension do not match model dimension")
        
        if x.device != self.pe.device:
            self.pe = self.pe.to(x.device)  # Move pe to the same device as x

        x = x + self.pe[:x.shape[1], :]
        return x

In [52]:
pe = PositionalEncoder(max_seq_len=10, d_model=5)
pe.show_pe()

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  2.5116e-02,  9.9968e-01,  6.3096e-04],
        [ 9.0930e-01, -4.1615e-01,  5.0217e-02,  9.9874e-01,  1.2619e-03],
        [ 1.4112e-01, -9.8999e-01,  7.5285e-02,  9.9716e-01,  1.8929e-03],
        [-7.5680e-01, -6.5364e-01,  1.0031e-01,  9.9496e-01,  2.5238e-03],
        [-9.5892e-01,  2.8366e-01,  1.2526e-01,  9.9212e-01,  3.1548e-03],
        [-2.7942e-01,  9.6017e-01,  1.5014e-01,  9.8866e-01,  3.7857e-03],
        [ 6.5699e-01,  7.5390e-01,  1.7493e-01,  9.8458e-01,  4.4167e-03],
        [ 9.8936e-01, -1.4550e-01,  1.9960e-01,  9.7988e-01,  5.0476e-03],
        [ 4.1212e-01, -9.1113e-01,  2.2415e-01,  9.7455e-01,  5.6786e-03]])


## True positional encoding obtained from the paper
Positional Encodings (max_seq_len=10, dim_model=5):<br>
[[ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.00000000e+00 0.00000000e+00]<br>
 [ 8.41470985e-01  5.40302306e-01  2.51162229e-02  9.99684538e-01 6.30957303e-04]<br>
 [ 9.09297427e-01 -4.16146837e-01  5.02165994e-02  9.98738351e-01 1.26191435e-03]<br>
 [ 1.41120008e-01 -9.89992497e-01  7.52852930e-02  9.97162035e-01 1.89287090e-03]<br>
 [-7.56802495e-01 -6.53643621e-01  1.00306487e-01  9.94956586e-01 2.52382670e-03]<br>
 [-9.58924275e-01  2.83662185e-01  1.25264396e-01  9.92123395e-01 3.15478149e-03]<br>
 [-2.79415498e-01  9.60170287e-01  1.50143272e-01  9.88664249e-01 3.78573502e-03]<br>
 [ 6.56986599e-01  7.53902254e-01  1.74927419e-01  9.84581331e-01 4.41668705e-03]<br>
 [ 9.89358247e-01 -1.45500034e-01  1.99601200e-01  9.79877217e-01 5.04763732e-03]<br>
 [ 4.12118485e-01 -9.11130262e-01  2.24149048e-01  9.74554875e-01 5.67858558e-03]]<br>

In [53]:
class FeedForwardNN(torch.nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.ff1 = torch.nn.Linear(self.d_model, self.d_ff)
        self.act = torch.nn.GELU()
        self.ff2 = torch.nn.Linear(self.d_ff, self.d_model)

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(0)
            print("Warning: batch size not present")

        if x.shape[2]!=self.d_model:
            raise Exception("Token dimension do not match model dimension")
        
        x = self.ff1(x)
        x = self.act(x)
        x = self.ff2(x)

        return x

In [54]:
class LayerNorm(torch.nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.d_model = d_model
        self.layerNorm = torch.nn.LayerNorm(d_model)

    def forward(self, x):
        x = self.layerNorm(x)
        return x

In [55]:
class SelfAttentionHead(torch.nn.Module):
    def __init__(self, d_model, d_k, max_seq_len):
        super().__init__()
        self.d_model = d_model
        self.d_k = d_k
        self.max_seq_len = max_seq_len
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.W_q = torch.nn.Linear(in_features=d_model, out_features=d_k)
        self.W_k = torch.nn.Linear(in_features=d_model, out_features=d_k)
        self.W_v = torch.nn.Linear(in_features=d_model, out_features=d_k)

        self.W_O = torch.nn.Linear(in_features = d_k, out_features=d_model)
        
    def forward(self, Q, K, V, mask = False):
        if len(Q.shape)==2:
            Q.unsqueeze(0)
            print("Warning: batch size not present")

        if len(K.shape)==2:
            K.unsqueeze(0)
            print("Warning: batch size not present")

        if len(V.shape)==2:
            V.unsqueeze(0)
            print("Warning: batch size not present")

        if Q.shape[1] > self.max_seq_len or V.shape[1]> self.max_seq_len or K.shape[1]>self.max_seq_len:
            raise Exception("Number of tokens exceed max sequence length")

        if Q.shape[2] != self.d_model or K.shape[2] != self.d_model or V.shape[2]!=self.d_model:
            raise Exception("Tokens dimension do not match model dimension")

        Q = self.W_q(Q)
        K = self.W_k(K)
        V = self.W_v(V)

        logits = (torch.matmul(Q, torch.transpose(K, 1, 2)) )/ (self.d_k ** 0.5)
        if mask is True:
            ones = torch.ones(Q.shape[0], Q.shape[1], Q.shape[1]).to(Q.device)
            mask = torch.tril(ones)
            logits = logits.masked_fill(mask == 0, -float('1e9'))

        scores = torch.nn.functional.softmax(logits, dim = 1)
        attention = torch.matmul(scores, V)

        output = self.W_O(attention)
        
        return output

In [56]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, num_heads, d_model, d_k, max_seq_len):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.d_k=d_k
        self.max_seq_len = max_seq_len

        self.heads = torch.nn.ModuleList([SelfAttentionHead(d_model=self.d_model, d_k=self.d_k, max_seq_len=self.max_seq_len) for _ in range(num_heads)])

        self.W_O = torch.nn.Linear(num_heads*self.d_model, self.d_model)

    def forward(self, Q, K, V, mask = False):
        if len(Q.shape)==2:
            Q.unsqueeze(0)
            print("Warning: batch size not present")

        if len(K.shape)==2:
            K.unsqueeze(0)
            print("Warning: batch size not present")

        if len(V.shape)==2:
            V.unsqueeze(0)
            print("Warning: batch size not present")

        if Q.shape[1] > self.max_seq_len or V.shape[1]> self.max_seq_len or K.shape[1]>self.max_seq_len:
            raise Exception("Number of tokens exceed max sequence length")

        if Q.shape[2] != self.d_model or K.shape[2] != self.d_model or V.shape[2]!=self.d_model:
            raise Exception("Tokens dimension do not match model dimension")

        head_outputs = [head(Q, K, V, mask) for head in self.heads]
        concatenated = torch.cat(head_outputs, dim=-1)
        return self.W_O(concatenated)

In [57]:
class EncoderBlock(torch.nn.Module):
    def __init__(self, num_heads, d_model, d_k, d_ff, max_seq_len):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.d_k = d_k
        self.d_ff = d_ff
        self.max_seq_len = max_seq_len

        self.multiHeadAttention = MultiHeadAttention(num_heads=self.num_heads, d_model=self.d_model, d_k=self.d_k, max_seq_len=self.max_seq_len)
        self.layerNorm = LayerNorm(d_model=self.d_model)
        self.ffnn = FeedForwardNN(d_model=self.d_model, d_ff=self.d_ff)

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(0)
            print("Warning: batch size not present")

        if x.shape[2]!=self.d_model:
            raise Exception("Token dimension do not match model dimension")

        y = self.multiHeadAttention(x, x, x, False)
        
        x = x + y
        x = self.layerNorm(x)
        
        y = self.ffnn(x)
        x = x + y
        x = self.layerNorm(x)
        
        return x

In [58]:
class DecoderBlock(torch.nn.Module):
    def __init__(self, num_heads, d_model, d_k, d_ff, max_seq_len):
        super().__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.d_k = d_k
        self.d_ff = d_ff
        self.max_seq_len = max_seq_len


        self.maskedMultiHeadAttention = MultiHeadAttention(num_heads=self.num_heads, d_model=self.d_model, d_k=self.d_k, max_seq_len=self.max_seq_len)
        self.multiHeadAttention = MultiHeadAttention(num_heads=self.num_heads, d_model=self.d_model, d_k=self.d_k, max_seq_len=self.max_seq_len)
        self.layerNorm = LayerNorm(d_model=self.d_model)
        self.ffnn = FeedForwardNN(d_model=self.d_model, d_ff=self.d_ff)

    def forward(self, Q, K, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(0)
            print("Warning: batch size not present")

        if x.shape[2]!=self.d_model:
            raise Exception("Token dimension do not match model dimension")
        
        if Q.shape[2]!=self.d_model or K.shape[2]!=self.d_model:
            raise Exception("Token dimension do not match model dimension")
        
        y = self.maskedMultiHeadAttention(x, x, x, mask=True)
        x = x + y
        x = self.layerNorm(x)

        y = self.multiHeadAttention(Q, K, x, mask=False)
        x = x + y
        x = self.layerNorm(x)

        y = self.ffnn(x)
        x = x + y
        x = self.layerNorm(x)

        return x

In [59]:
class Transformer(torch.nn.Module):
    def __init__(self, num_encoder_heads, num_decoder_heads, d_model, d_k, d_enc_ff, d_dec_ff, max_seq_len, num_encoder_blocks, num_decoder_blocks, word_embedder = None):
        super().__init__()
        self.num_encoder_heads = num_encoder_heads
        self.num_decoder_heads = num_decoder_heads
        self.d_model = d_model
        self.d_k = d_k
        self.d_enc_ff = d_enc_ff
        self.d_dec_ff = d_dec_ff
        self.max_seq_len = max_seq_len
        self.num_encoder_blocks = num_encoder_blocks
        self.num_decoder_blocks = num_decoder_blocks
        self.word_embedder = word_embedder

        self.pe = PositionalEncoder(max_seq_len=self.max_seq_len, d_model=self.d_model)
        self.encoders = torch.nn.ModuleList([EncoderBlock(d_model=self.d_model, d_k=self.d_k, max_seq_len=self.max_seq_len, d_ff=self.d_enc_ff, num_heads=self.num_encoder_heads) for _ in range(self.num_encoder_blocks)])
        self.decoders = torch.nn.ModuleList([DecoderBlock(d_model=self.d_model, d_k=self.d_k, max_seq_len=self.max_seq_len, d_ff=self.d_dec_ff, num_heads=self.num_decoder_heads) for _ in range(self.num_decoder_blocks)])
    
    def forward(self, x_enc, x_dec):
        if len(x_enc.shape) == 2 or len(x_dec.shape)==2:
            x = x.unsqueeze(0)
            print("Warning: batch size not present")
        
        if x_enc.shape[1] > self.max_seq_len or x_dec.shape[1]>self.max_seq_len:
            raise Exception("Number of tokens exceeds max sequence length")
        
        if self.word_embedder is not None:
            x_enc = self.word_embedder(x_enc)
            x_dec = self.word_embedder(x_dec)

        if x_enc.shape[2]!=self.d_model or x_dec.shape[2]!=self.d_model:
            raise Exception("Token dimension do not match model dimension")

        x_enc = self.pe(x_enc)

        for block in self.encoders:
            x_enc = block(x_enc)

        x_dec = self.pe(x_dec)
        for block in self.decoders:
            x_dec = block(Q=x_enc, K=x_enc, x=x_dec)  # Q, K from encoder, V from decoder
        
        return x_dec

## From now on, tests

In [60]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NUM_ENC_HEADS = 3
NUM_DEC_HEADS = 3
MAX_SEQ_LEN = 500
D_MODEL = 512
D_K = 128
D_ENC_FF = 256
D_DEC_FF = 256
NUM_ENC_BLOCKS = 2
NUM_DEC_BLOCKS = 2
BATCH_SIZE = 16

model = Transformer(
    num_encoder_heads = NUM_ENC_HEADS,
    num_decoder_heads = NUM_DEC_HEADS,
    d_model = D_MODEL,
    d_k = D_K,
    d_enc_ff = D_ENC_FF,
    d_dec_ff = D_DEC_FF,
    max_seq_len = MAX_SEQ_LEN,
    num_encoder_blocks = NUM_ENC_BLOCKS,
    num_decoder_blocks = NUM_DEC_BLOCKS
    ).to(DEVICE)

model = torch.nn.DataParallel(model, device_ids=[0,1])

In [61]:
seq_len = 10
d_model = 512

input_tensor = torch.randn(1, seq_len, d_model).to(DEVICE)
output = model(x_dec = input_tensor, x_enc = input_tensor)

assert output.shape == (1, seq_len, d_model), f"Unexpected output shape: {output.shape}"
print("Shape test passed!")

Shape test passed!


In [62]:
#TODO: need more tests!